![](image.jpg)


Dive into the heart of data science with a project that combines healthcare insights and predictive analytics. As a Data Scientist at a top Health Insurance company, you have the opportunity to predict customer healthcare costs using the power of machine learning. Your insights will help tailor services and guide customers in planning their healthcare expenses more effectively.

## Dataset Summary

Meet your primary tool: the `insurance.csv` dataset. Packed with information on health insurance customers, this dataset is your key to unlocking patterns in healthcare costs. Here's what you need to know about the data you'll be working with:

## insurance.csv
| Column    | Data Type | Description                                                      |
|-----------|-----------|------------------------------------------------------------------|
| `age`       | int       | Age of the primary beneficiary.                                  |
| `sex`       | object    | Gender of the insurance contractor (male or female).             |
| `bmi`       | float     | Body mass index, a key indicator of body fat based on height and weight. |
| `children`  | int       | Number of dependents covered by the insurance plan.              |
| `smoker`    | object    | Indicates whether the beneficiary smokes (yes or no).            |
| `region`    | object    | The beneficiary's residential area in the US, divided into four regions. |
| `charges`   | float     | Individual medical costs billed by health insurance.             |



A bit of data cleaning is key to ensure the dataset is ready for modeling. Once your model is built using the `insurance.csv` dataset, the next step is to apply it to the `validation_dataset.csv`. This new dataset, similar to your training data minus the `charges` column, tests your model's accuracy and real-world utility by predicting costs for new customers.

## Let's Get Started!

This project is your playground for applying data science in a meaningful way, offering insights that have real-world applications. Ready to explore the data and uncover insights that could revolutionize healthcare planning? Let's begin this exciting journey!

In [136]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np

# Loading the insurance dataset
insurance_data_path = 'insurance.csv'
insurance = pd.read_csv(insurance_data_path)
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19.0,female,27.900,0.0,yes,southwest,16884.924
1,18.0,male,33.770,1.0,no,Southeast,1725.5523
2,28.0,male,33.000,3.0,no,southeast,$4449.462
3,33.0,male,22.705,0.0,no,northwest,$21984.47061
4,32.0,male,28.880,0.0,no,northwest,$3866.8552


In [137]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1272 non-null   float64
 1   sex       1272 non-null   object 
 2   bmi       1272 non-null   float64
 3   children  1272 non-null   float64
 4   smoker    1272 non-null   object 
 5   region    1272 non-null   object 
 6   charges   1284 non-null   object 
dtypes: float64(3), object(4)
memory usage: 73.3+ KB


In [138]:
# cleaning data 
insurance_clean = insurance.copy()
insurance_clean = insurance_clean[insurance_clean["age"] > 0]
insurance_clean["sex"] = insurance_clean["sex"].replace({"M": "male", "F":"female", "man":"male", "woman":"female"})
insurance_clean.loc[insurance_clean["children"] < 0, "children"] = 0
insurance_clean["region"] = insurance_clean["region"].str.lower()
insurance_clean["charges"] = insurance_clean["charges"].replace({"\$":""}, regex=True).astype("float")
insurance_clean = insurance_clean.dropna()
insurance_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1149 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1149 non-null   float64
 1   sex       1149 non-null   object 
 2   bmi       1149 non-null   float64
 3   children  1149 non-null   float64
 4   smoker    1149 non-null   object 
 5   region    1149 non-null   object 
 6   charges   1149 non-null   float64
dtypes: float64(4), object(3)
memory usage: 71.8+ KB


In [139]:
# preprocessing data
scaler = StandardScaler()
categorical = ["sex", "smoker", "region"]
numerical = ["age", "bmi", "children"]
X = insurance_clean.drop(columns="charges")
y = insurance_clean["charges"]
X_cat = pd.get_dummies(X[categorical], drop_first=True)
X_pp = pd.concat([X_cat, X[numerical]], axis=1)
X_scaled = scaler.fit_transform(X_pp)

In [140]:
# deploy model and cross-validate
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

lin = LinearRegression()
steps = [("scaler", scaler), ("lin", lin)]
pipeline = Pipeline(steps)
pipeline.fit(X_scaled, y)
mse_scores = -cross_val_score(pipeline, X_scaled, y, cv=5, scoring="neg_mean_squared_error")
r2_scores = cross_val_score(pipeline, X_scaled, y, cv=5, scoring="r2")
mse_score = np.mean(mse_scores)
r2_score = np.mean(r2_scores)
print("Mean_mse:", mse_score)
print("Mean_r2:", r2_score)

Mean_mse: 37431001.52191916
Mean_r2: 0.7450511466263761


In [141]:
# Deploy on test set
validation = pd.read_csv("validation_dataset.csv")
validation_processed = pd.get_dummies(validation, columns=["sex", "smoker", "region"], drop_first=True)
validation_pred = pipeline.predict(validation_processed)
validation_data = pd.DataFrame(validation_pred, columns=["predicted_charges"])
validation_data = pd.concat([validation, validation_data], axis=1)
validation_data.loc[validation_data["predicted_charges"]<1000, "predicted_charges"] = 1000